In [1]:
import tensorflow as tf
import numpy as np

import keras
import pickle
import h5py
import time
import gzip

from tensorflow.python.ops import data_flow_ops
from tensorflow.keras.layers import Embedding

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load the input vectors

In [0]:
with gzip.open('./drive/My Drive/CoE202TermProject/datasets/train.chunk.pickle', 'rb') as f:
    traindata = pickle.load(f)

with gzip.open('./drive/My Drive/CoE202TermProject/datasets/train.image.feats.pickle', 'rb') as f:
    train_images = pickle.load(f)

train_product = traindata['product'][:]
train_label = traindata['label'][:]


with gzip.open('./drive/My Drive/CoE202TermProject/datasets/valid.chunk.pickle', 'rb') as f:
    validdata = pickle.load(f)

with gzip.open('./drive/My Drive/CoE202TermProject/datasets/valid.image.feats.pickle', 'rb') as f:
    valid_images = pickle.load(f)

valid_product = validdata['product'][:]
valid_label = validdata['label'][:]

In [0]:
batch_size = 512
lr         = 1e-4
n_epochs   = 3
valid_freq = 3
n_classes  = np.max(train_label) + 1

### Placeholders

In [0]:
input_images = tf.placeholder(dtype=tf.float32, shape=(None, 2048), name='input_images')
input_prices = tf.placeholder(dtype=tf.float32, shape=(None, ), name='input_prices')
labels = tf.placeholder(dtype=tf.int32, shape=(None, ), name='labels')
is_train = tf.placeholder(dtype=tf.bool, name='is_train')

### Deep neural network

In [6]:
# Processing Images
x_images = tf.keras.layers.Dense(n_classes * 2)(input_images)
x_images = tf.layers.batch_normalization(x_images, training=is_train)
x_images = tf.nn.relu(x_images)

hidden1 = tf.keras.layers.Dense(n_classes)(x_images)
hidden1 = tf.layers.batch_normalization(hidden1, training=is_train)
hidden1 = tf.nn.relu(hidden1)

# Normalizing Prices
x_prices = tf.reshape(input_prices, (-1, 1))
x_prices = tf.layers.batch_normalization(x_prices, training=is_train)

# Concatenating Images and Prices
x_concat = tf.keras.layers.Concatenate(axis=1)([hidden1, x_prices])

# Hidden Layer
hidden2 = tf.keras.layers.Dense(n_classes)(x_concat)
hidden2 = tf.layers.batch_normalization(hidden2, training=is_train)
hidden2 = tf.nn.relu(hidden2)

# Output
logits = tf.keras.layers.Dense(n_classes)(hidden2)

# Softmax
preds = tf.nn.softmax(logits, name='predictions')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.


### Loss funciton & Optimizer

In [7]:
# Softmax cross entropy loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=tf.one_hot(labels, n_classes), logits=logits)

# Weight decay
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([loss] + reg_losses, name='total_loss')

# Optimizer
optm = tf.train.AdamOptimizer(lr)
train_op = optm.minimize(loss, global_step=tf.train.get_global_step(), name='step_update')
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
train_op = tf.group([train_op, update_ops])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Accuracy

In [0]:
top1_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=1)
top1_acc = tf.identity(top1_acc, name='top1_acc')

top5_acc = tf.keras.metrics.top_k_categorical_accuracy(y_true=tf.one_hot(labels, n_classes),
                                                        y_pred=preds, k=5)
top5_acc = tf.identity(top5_acc, name='top5_acc')

### Batch generator

In [0]:
def generator(mode='training'):
    if mode == 'training':
        products = train_product
        labels = train_label
        images = train_images

    elif mode == 'valid':
        products = valid_product
        labels = valid_label
        images = valid_images
    
    elif mode == 'test':
        products = test_product
        labels = None
        images = test_images

    n_data = len(products)
    indices = np.arange(n_data)
    np.random.shuffle(indices)
    
    for start_idx in range(0, n_data, batch_size):
        if start_idx + batch_size <= n_data:
            excerpt = indices[start_idx: start_idx + batch_size]

            if labels is not None:
                yield list(zip(*products[excerpt, :])), \
                    images[excerpt, :], \
                    labels[excerpt]
            
            else:
                yield list(zip(*products[excerpt, :])), \
                    images[excerpt, :]


### Training session

In [10]:
import tqdm
import math

with tf.Session() as sess:
    tic = time.time()
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epochs):
        print("\n\nEpoch {0:03d} / {1:03d}\n".format(epoch, n_epochs))
        training_loss = []
        for b_product, b_images, b_label in tqdm.tqdm(
            generator(mode='training'),
            total=math.ceil(len(train_label) / batch_size)
        ):
            feed_dict = {
                input_prices: b_product[6],
                input_images: b_images,
                labels: b_label,
                is_train: True
            }
            
            _, train_loss = sess.run(
                [train_op, loss],
                feed_dict=feed_dict
            )

            training_loss.append(train_loss)

        toc = time.time()
        print(
            "[*] TRAIN Loss {0:.4f} | Time {1:.2f}s"
            .format(np.mean(training_loss), toc - tic)
        )
        
        if (epoch+1) % valid_freq == 0:
            top_1, top_5 = [], []
            for b_product, b_images, b_label in generator(mode='valid'):
                feed_dict = {
                    input_prices: b_product[6],
                    input_images: b_images,
                    labels: b_label,
                    is_train: True
                }
                    
                t1_acc, t5_acc = sess.run(
                    [top1_acc, top5_acc],
                    feed_dict=feed_dict
                )

                top_1.append(t1_acc)
                top_5.append(t5_acc)

            print(
                "[*] VALIDATION Top-1 Acc: {0:.4f} | Top-5 Acc: {1:.4f}"
                .format(np.mean(top_1), np.mean(top_5))
            )

    saver.save(sess, './drive/My Drive/CoE202TermProject/models/models')

  0%|          | 0/1563 [00:00<?, ?it/s]



Epoch 000 / 003



  0%|          | 2/1563 [00:00<02:01, 12.86it/s]

[*] TRAIN Loss 2.1437 | Time 110.52s


Epoch 001 / 003



  0%|          | 2/1563 [00:00<01:57, 13.32it/s]

[*] TRAIN Loss 1.1751 | Time 217.06s


Epoch 002 / 003



100%|█████████▉| 1562/1563 [01:46<00:00, 14.80it/s]


[*] TRAIN Loss 0.8341 | Time 323.25s
[*] VALIDATION Top-1 Acc: 0.6886 | Top-5 Acc: 0.9003


## Test process
### Load the test data

In [0]:
# test data
with gzip.open('./drive/My Drive/CoE202TermProject/datasets/test.chunk.pickle', 'rb') as f:
    testdata = pickle.load(f)

with gzip.open('./drive/My Drive/CoE202TermProject/datasets/valid.image.feats.pickle', 'rb') as f:
    test_images = pickle.load(f)

test_product  = testdata['product'][:]
pids          = testdata['pids'][:]

### Batch generator for test dataset

In [12]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('./drive/My Drive/CoE202TermProject/models/models.meta')
    saver.restore(sess, tf.train.latest_checkpoint('./drive/My Drive/CoE202TermProject/models/'))
    DNN = tf.get_default_graph()

    preds = []
    for b_product, b_images in generator(mode='test'):
        feed_dict = {
            DNN.get_tensor_by_name('input_images:0'): b_images,
            DNN.get_tensor_by_name('input_prices:0'): b_product[6],
            DNN.get_tensor_by_name('is_train:0'): False
        }

        pred = sess.run(DNN.get_tensor_by_name('predictions:0'), feed_dict=feed_dict)
        preds.extend(pred)


INFO:tensorflow:Restoring parameters from ./drive/My Drive/CoE202TermProject/models/models


### Save the submission files

In [0]:
# Indexing of predictions
argpreds = np.argmax(preds, axis=1)

# Load label dictionary
with open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle', 'rb') as f:
    y_dict = pickle.load(f)
# y_dict = pickle.loads(open('./drive/My Drive/CoE202TermProject/datasets/y_vocab.pickle').read())

# Inverse label dictionary
inv_y_dict = dict((y,x) for x,y in y_dict.items())
submissions = [inv_y_dict[argpred] for argpred in argpreds]

# Write the results to 'submissions.csv'
f = open('./drive/My Drive/CoE202TermProject/submissions.csv', 'w')
for i, j in zip(pids, submissions):
    line = '{},{}\n'.format(i,j)
    f.write(line)
f.close()

You should submit the 'submissions.csv' file and 4 tf.save files ('checkpoint', 'dnn_models.data', 'dnn_models.index', 'dnn_models.meta') in models folder